In [1]:
import aiohttp
import asyncio
import bs4
import datetime
from io import BytesIO
import os.path
import pandas as pd
import PyPDF2
import requests
from tqdm.contrib.telegram import tqdm

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar, LTTextLineHorizontal
from PyPDF2 import PdfReader

from urllib.parse import unquote
# from transliterate import to_latin
import os

import time
from datetime import datetime
import pytz
import logging



In [2]:
TOKEN = os.getenv('TOKEN')
CHAT_ID = os.getenv('CHAT_ID')

In [3]:
NEWS_URL = "https://kun.uz/uz/news/list?f=latest&next={unix_time}"

In [4]:
BASE_URL = "https://kun.uz" 

In [5]:
def get_utc(hour, minute):

    utc = int(datetime(datetime.now().year, datetime.now().month, datetime.now().day, int(hour), int(minute)).timestamp()) - (5 * 3600)
    
    return utc

In [6]:
# global_links = []
# global_dates = []


def daily_scrapping_urls(unix_time,links=[],dates=[]):
    while True:
        try:
            resp = requests.get(NEWS_URL.format(unix_time=unix_time))
            if resp.status_code == 200:
                data = resp.text
                soup = bs4.BeautifulSoup(data)
                selected_links = soup.find('div', class_='daily-news max-w').find_all('a')
                

                links.extend([BASE_URL+link['href'] for link in selected_links])
                dates.extend([link.find('p', class_="news-date").text for link in selected_links])


                dates1 = ["24:00"] + dates

                for i in range(len(dates)):
                    if dates1[i] < dates[i]:
#                         print(dates1[i], dates[i], i)
                        return ([links, dates], i)

                hour, minute = dates[-1].split(":")
                utc = get_utc(hour, minute)
                return daily_scrapping_urls(utc,links,dates)
            elif resp.status_code != 404:
                time.sleep(60) # 1 minute
        except Exception as e:
            print(e)

In [7]:
def daily_scrapping_news():
    try:
        results = []

        unix_time = int(time.time()) 
        values = daily_scrapping_urls(unix_time)

        values = {"links":values[0][0][:values[1]], "dates":values[0][1][:values[1]]}



        for link in values['links']:
            resp = requests.get(link)

            if resp.status_code == 200:
                data = resp.text
                soup = bs4.BeautifulSoup(data)
                title = soup.find('h1', class_='single-header__title').text
                content = soup.find('div', class_='single-content').text

                results.append({
                                "title": title.strip(),
                                "content": content.strip()
                            })


            elif resp.status_code != 404:
                print("404")
                time.sleep(60) # 1 minute

        return results
    except Exception as e:
        print(e)

In [8]:
def get_values_df():
    values = daily_scrapping_news()
    return pd.DataFrame(values)

In [9]:
def update_df():
    try:
        path = './DailyKunUz.pickle'
        check_file = os.path.exists(path)
        if check_file:
            df = pd.read_pickle(path)
        else:
            df = pd.DataFrame()

        current_date = datetime.now(pytz.timezone('Asia/Tashkent'))

        adding_df = get_values_df()
        adding_df.to_pickle(f'daily pickles/{current_date.day}-{current_date.month}-{current_date.year}-kunuz.pickle')

        df = pd.concat([df, adding_df], axis=0, ignore_index=True)
        df.to_pickle(path)

        logging.basicConfig(filename='scheduler.log', filemode='a', format='%(name)s - %(levelname)s - %(message)s')
        logging.getLogger().setLevel(logging.INFO)
        logging.info(f"KUN.uz {current_date} COMPLETE!")

        print(current_date, "COMPLETE!")
    except Exception as e:
        print(e)

In [ ]:
# Schedule Library imported
import schedule
import time


schedule.every().day.at("18:58").do(update_df) # 23:58 UTC+5

# Loop so that the scheduling task
# keeps on running all time.
while True:
 
    # Checks whether a scheduled task
    # is pending to run or not
    schedule.run_pending()
    time.sleep(1)

# DIPLOM 

In [48]:
diplom = [[66, 3], [95, 5], [168, 5], [143, 5], [446, 5], [64, 5], [142, 4], [128, 5], [68, 5], [204, 4], [104, 5], [128, 5], [271, 5], 
          [271, 5], [271, 5], [71, 5], [335, 5], [414, 5], [316, 4], [145, 5], [78, 4], [78, 5], [150, 4], [52,4], [271, 5], [142, 4], 
          [71, 5], [120, 4], [412, 4], [343, 5], [271, 5], [143, 5], [143, 5], [107, 4], [271, 5], [356, 5]]

In [49]:
practice = [[756, 4], [324, 5], [432, 5], [1188, 4]]

In [50]:
diplom1 = diplom+practice

In [51]:
diplom1

[[66, 3],
 [95, 5],
 [168, 5],
 [143, 5],
 [446, 5],
 [64, 5],
 [142, 4],
 [128, 5],
 [68, 5],
 [204, 4],
 [104, 5],
 [128, 5],
 [271, 5],
 [271, 5],
 [271, 5],
 [71, 5],
 [335, 5],
 [414, 5],
 [316, 4],
 [145, 5],
 [78, 4],
 [78, 5],
 [150, 4],
 [52, 4],
 [271, 5],
 [142, 4],
 [71, 5],
 [120, 4],
 [412, 4],
 [343, 5],
 [271, 5],
 [143, 5],
 [143, 5],
 [107, 4],
 [271, 5],
 [356, 5],
 [756, 4],
 [324, 5],
 [432, 5],
 [1188, 4]]

In [63]:
a = 0

In [64]:
for mark in diplom1:
    a += (mark[1] -1) * mark[0]

In [65]:
b = sum([mark[0] for mark in diplom1])

In [66]:
a, b

(34433, 9558)

In [67]:
a/b

3.602531910441515